In [252]:
import pandas as pd
import numpy as np
import gspread
import datetime as DT
from gspread_pandas import Spread
from operator import add


In [253]:
from plaid import Client

# Извлекаем данные из Plaid,используем ключ, соответствующий выбраному банку, выбираем дату 

client = Client(client_id='5ed6321c2aafa90012d29de8', secret='58965fb8be508062c7ce1438ae30ff', public_key='9b745bacdb0623990519cd2f193c09', environment='sandbox')

access_token = 'access-sandbox-b80dde63-2e00-4290-82ac-3d8b25ee19d4'
start_date = '2019-01-01'
end_date = '2020-05-30'
response = client.Transactions.get(access_token, start_date= start_date, end_date= end_date)
transactions = response['transactions']

# the transactions in the response are paginated, so make multiple calls while increasing the offset to
# retrieve all transactions
while len(transactions) < response['total_transactions']:
    response = client.Transactions.get(access_token, start_date= start_date, end_date= end_date,
                                       offset=len(transactions)
                                      )
    transactions.extend(response['transactions'])

In [254]:
# Проверяем получение
len(transactions)

275

In [255]:
# Создаем листы с данными по каждому интересуещему нас столбцу

category1 = []
category2 = []

for i in range(len(transactions)):
    category1.append(transactions[i]['category'][0])
    if len(transactions[i]['category']) > 1:
        category2.append(transactions[i]['category'][1])
    else:
        category2.append('Missing')

date = []

for i in range(len(transactions)):
    date.append(DT.datetime.strptime(transactions[i]['date'], '%Y-%m-%d').date())        

amount = []

for i in range(len(transactions)):
    amount.append(int(transactions[i]['amount']))
    
status = []    
    
for i in range(len(transactions)):
    if amount[i] < 0:
        status.append('Income')
    else:
        status.append('Expense')

In [256]:
# Создаем DataFrame с сырыми данными 
data = { 'Amount': amount, 'Category1': category1, 'Category2': category2, 
                     'Date': date, 'Income/Expense':status}
df = pd.DataFrame (data)

df['Amount'] = df['Amount']*(-1) # Меняем знак тк в выгрузке значения инверсированы



In [257]:
# Создаем листы с наименованиями строк, чтобы создать многоуровневую таблицу
# Листы одинаковой длинны

status_tup = []
category1_tup = []
category2_tup = []

for status in ['Income', 'Expense']:
    raws1 = df[df['Income/Expense'] == status]['Category1'].unique().tolist()
    raws1.append('Total')
    for cat1 in raws1:
        if cat1 == 'Total':
            category2_tup.append('')
            category1_tup.append('Total')
            status_tup.append(status)
        else:
            raws2 = df[(df['Category1'] == cat1) & (df['Income/Expense'] == status)]['Category2'].unique().tolist()
            raws2.append('Total')
            
            for cat2 in raws2:
                category1_tup.append(cat1)
                status_tup.append(status)
                category2_tup.append(cat2)


In [258]:
# Создаем словарь для месяцев 
month_dic = {'1' :'Janauary',
         '2':'February',
         '3':'March',
         '4':'April',
         '5':'May',
         '6':'June',
         '7':'July',
         '8':'August',
         '9':'September',
         '10':'October',
         '11':'November',
         '12':'December'}

# Переводим дату в тнужный формат
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Считаем сколько месяцев между датами
months = end_date.month - start_date.month + 12*(end_date.year - start_date.year) +1 

# Создаем columns с месяцами и годами, чтобы потом использовать в таблице

columns = []
month = start_date.month
year = start_date.year
for i in range(months):
    if month == 13:
        month = 1
        year = year + 1
    columns.append(month_dic[str(month)]+ ' ' + str(year) )
    month = month +1 

In [259]:
# Функция для суммирования значений по категории2 и месяцам
# на выходе получаем вектор 1 х кол. месяцев с значениями для категории2 и статуса(income/expense)
def func(raws2, status):
    vector = np.zeros(len(columns), dtype= int)
    for cat2 in raws2:
        res = df[(df['Category2'] == cat2) & (df['Income/Expense'] == status)].set_index('Date').groupby(pd.Grouper(freq='M'))['Amount'].sum().reset_index()
        vector = list( map(add, vector, res['Amount'].tolist()))
    return(vector)

In [260]:
# Создаем словарь, где keys - месяца, а значения - столбцы c суммами
data_new = {}
for i in columns:
    data_new[str(i)] = [] 

df['Date'] = pd.to_datetime(df['Date'])  #делаем правильный формат даты, чтобы группировать

for status, cat1, cat2 in zip(status_tup, category1_tup, category2_tup): #идём перебором по всем строкам в категории2
    
    if cat2 == 'Total': #вставляем значения для Total по каждой категории1
        
        cat1_sub = df[(df['Category1'] == cat1) & (df['Income/Expense'] == status)]['Category2'].unique().tolist()
        vector = func(cat1_sub, status)
        
        for i, j in zip(columns, range(len(columns))): #вставляем 
            data_new[str(i)].append(vector[j])
    
    if cat2 == '': # вставляем Total по Income/Expense
        dic_for_formating = {status: len(data_new[columns[0]]) }
        
        status_sub = df[df['Income/Expense'] == status]['Category2'].unique().tolist()
        vector = func(status_sub, status)
        
        for i, j in zip(columns, range(len(columns))):
            data_new[str(i)].append(vector[j])
    
    if cat2 != 'Total' and cat2 != '': # Вставляем значения для всех остальных категорий2
        res = df[(df['Category2'] == cat2) & (df['Income/Expense'] == status)].set_index('Date').groupby(pd.Grouper(freq='M'))['Amount'].sum().reset_index()
        
        for i, j in zip(columns, range(len(columns))):
            data_new[str(i)].append(res['Amount'][j])
          

In [261]:
# Собираем в кучу первую версию cashflow таблицы
tuples = list(zip(*[status_tup, category1_tup, category2_tup]))

index = pd.MultiIndex.from_tuples(tuples, names=['Income/Expense', 'Category1', 'Category2'])

df_new = pd.DataFrame(data_new, index=index, columns= columns)
df_new

Janauary 2019  \
Income/Expense Category1      Category2                                       
Income         Travel         Airlines and Aviation Services            500   
                              Total                                     500   
               Transfer       Credit                                      4   
                              Total                                       4   
               Total                                                    504   
Expense        Travel         Airlines and Aviation Services           -500   
                              Taxi                                      -11   
                              Total                                    -511   
               Food and Drink Restaurants                             -1605   
                              Total                                   -1605   
               Payment        Missing                                 -2078   
                              Credit Card                               -25   
                              Total                                   -2103   
               Shops          Sporting Goods                           -500   
                              Total                                    -500   
               Transfer       Debit                                   -5850   
                              Deposit                                 -1000   
                              Total                                   -6850   
               Recreation     Gyms and Fitness Centers                  -78   
                              Total                                     -78   
               Total                                                 -11647   

                                                              February 2019  \
Income/Expense Category1      Category2                                       
Income         Travel         Airlines and Aviation Services            500   
                              Total                                     500   
               Transfer       Credit                                      4   
                              Total                                       4   
               Total                                                    504   
Expense        Travel         Airlines and Aviation Services           -500   
                              Taxi                                      -11   
                              Total                                    -511   
               Food and Drink Restaurants                              -105   
                              Total                                    -105   
               Payment        Missing                                     0   
                              Credit Card                               -25   
                              Total                                     -25   
               Shops          Sporting Goods                              0   
                              Total                                       0   
               Transfer       Debit                                   -5850   
                              Deposit                                 -1000   
                              Total                                   -6850   
               Recreation     Gyms and Fitness Centers                  -78   
                              Total                                     -78   
               Total                                                  -7569   

                                                              March 2019  \
Income/Expense Category1      Category2                                    
Income         Travel         Airlines and Aviation Services         500   
                              Total                                  500   
               Transfer       Credit                                   4   
                              Total           

In [262]:
# Начинаем готовить данные для горизонтальной суммарную отчетности
# Суммируем данные по Total 
df1 = df_new.loc[('Income','Total', '')]
df2 = df_new.loc[('Expense','Total', '')]
df_add = df1.add(df2, fill_value=0)

# Создаем словарь по месяцам с данными
new_row = {}
for i in columns:
    new_row[i] = df_add[i].astype(int)

# Расширяем словарь с основными данными
for i in columns:
    data_new[i].append(new_row[i])

In [263]:
# Удлинняем строковые листы 
status_tup.append('Grand Total')
category1_tup.append('Grand Total')
category2_tup.append('Grand Total')

# Строим конечную версию cashflow
tuples = list(zip(*[status_tup, category1_tup, category2_tup]))

index = pd.MultiIndex.from_tuples(tuples, names=['Income/Expense', 'Category1', 'Category2'])

df_new = pd.DataFrame(data_new, index=index, columns= columns)

# Вставляем вертикальную суммарную отчетность
df_new["Grand Total"] = df_new.sum(axis=1)
df_new

Janauary 2019  \
Income/Expense Category1      Category2                                       
Income         Travel         Airlines and Aviation Services            500   
                              Total                                     500   
               Transfer       Credit                                      4   
                              Total                                       4   
               Total                                                    504   
Expense        Travel         Airlines and Aviation Services           -500   
                              Taxi                                      -11   
                              Total                                    -511   
               Food and Drink Restaurants                             -1605   
                              Total                                   -1605   
               Payment        Missing                                 -2078   
                              Credit Card                               -25   
                              Total                                   -2103   
               Shops          Sporting Goods                           -500   
                              Total                                    -500   
               Transfer       Debit                                   -5850   
                              Deposit                                 -1000   
                              Total                                   -6850   
               Recreation     Gyms and Fitness Centers                  -78   
                              Total                                     -78   
               Total                                                 -11647   
Grand Total    Grand Total    Grand Total                            -11143   

                                                              February 2019  \
Income/Expense Category1      Category2                                       
Income         Travel         Airlines and Aviation Services            500   
                              Total                                     500   
               Transfer       Credit                                      4   
                              Total                                       4   
               Total                                                    504   
Expense        Travel         Airlines and Aviation Services           -500   
                              Taxi                                      -11   
                              Total                                    -511   
               Food and Drink Restaurants                              -105   
                              Total                                    -105   
               Payment        Missing                                     0   
                              Credit Card                               -25   
                              Total                                     -25   
               Shops          Sporting Goods                              0   
                              Total                                       0   
               Transfer       Debit                                   -5850   
                              Deposit                                 -1000   
                              Total                                   -6850   
               Recreation     Gyms and Fitness Centers                  -78   
                              Total                                     -78   
               Total                                                  -7569   
Grand Total    Grand Total    Grand Total                             -7065   

                                                              March 2019  \
Income/Expense Category1      Category2                                    
Income         Travel         Airlines and Aviation Services         500   
                              Total     

In [264]:
# Авторизируемся для gspread_pandas через OAth ID
s = Spread('1zGw_kyJ-UaajLnaZ0QsN712_ex9tvq2wzSrMhICrnH4')
s

<gspread_pandas.spread.Spread - 'User: 'ens.tolik@gmail.com', Spread: 'Cashflow', Sheet: 'Raw data''>

In [265]:
# Отправляем в google sheets сырые данные
s.df_to_sheet(df, start = 'A1', sheet = 'Raw data', replace = True, freeze_headers = 1)

In [266]:
# Отправляем в google sheets готовый cashflow
s.df_to_sheet(df_new, start = 'A1', sheet = 'Cashflow', replace = True, freeze_headers = 1,
              freeze_index = 1 )

In [267]:
# Подключаем API google sheets и наводим красоту

In [268]:
import httplib2
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials

In [269]:
CREDENTIALS_FILE = 'creds.json'
# ID Google Sheets документа 
spreadsheet_id = '1zGw_kyJ-UaajLnaZ0QsN712_ex9tvq2wzSrMhICrnH4'

# Авторизуемся и получаем service 
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    CREDENTIALS_FILE,
    ['https://www.googleapis.com/auth/spreadsheets',
     'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth)

In [270]:
# Заготовка для быстрого объединения
def req_merge(start, lenth, level):   
    request =  {'mergeCells': {                    
        'mergeType': 'MERGE_COLUMNS',
        'range': {
            'startRowIndex': start,
            'endRowIndex': start + lenth ,
            'sheetId': '815608726',
            'startColumnIndex': level - 1,
            'endColumnIndex': level
            }
    }}
    return(request)

In [271]:
# Собираем пакет с объединениями ячеек для отправки 
formating = []
# Словарь с длинной объединения по каждой категории1
dict_formating = {} 

for j in sorted(df['Income/Expense'].unique(), reverse = True):
    for i in df[df['Income/Expense'] == j]['Category1'].unique():
         dict_formating [j, i] = len(df_new.loc[(j,i)])
            
start = 1 
for j in sorted(df['Income/Expense'].unique(), reverse = True):
    for i in df[df['Income/Expense'] == j]['Category1'].unique():
        
        request = req_merge(start, dict_formating [j, i], 2 )
        start = start + dict_formating [j, i] 
        formating.append(request)
    start = start + 1



C:\Users\UserPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  


In [272]:
len1 = len(df_new.loc['Income'][columns[0]]) # длинна объединения стобца по Income
len2 = len(df_new.loc['Expense'][columns[0]])# длинна объединения стобца по Expense

formating.append(req_merge(1, len1, 1)) #объединения стобца по Income
formating.append(req_merge(len1 + 1, len2, 1)) #объединения стобца по Expense
formating.append({'mergeCells': {          #объединения строки по Grand Total          
        'mergeType': 'MERGE_ALL',
        'range': {
            'startRowIndex': len1 + len2 + 1,
            'endRowIndex': len1 + len2 + 2 ,
            'sheetId': '815608726',
            'startColumnIndex': 0,
            'endColumnIndex': 3
            }
    }})

# Отправляем пакет
service.spreadsheets().batchUpdate( 
    spreadsheetId=spreadsheet_id,
     body={'requests': formating}
).execute()

{'spreadsheetId': '1zGw_kyJ-UaajLnaZ0QsN712_ex9tvq2wzSrMhICrnH4',
 'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]}

In [274]:
# Меняем расположение в ячейках для первых 3 столбцов в таблице
align_reques = {'repeatCell': {
    'cell': {
      'userEnteredFormat': {
        'horizontalAlignment': "LEFT",
        'verticalAlignment': "TOP",
      }
    },
    'range': {
      'sheetId': 815608726,
      'startRowIndex': 0,
      'endRowIndex': len1 + len2 + 2,
      'startColumnIndex': 0,
      'endColumnIndex': 3,
    },
    'fields': "userEnteredFormat"
}}
service.spreadsheets().batchUpdate( 
    spreadsheetId=spreadsheet_id,
     body={'requests': align_reques}
).execute()

{'spreadsheetId': '1zGw_kyJ-UaajLnaZ0QsN712_ex9tvq2wzSrMhICrnH4',
 'replies': [{}]}